In [70]:
from initial_functions import fn_read_from_db, fn_write_to_db
import pandas as pd
from sqlalchemy import create_engine
from context_fixed_range_calc import *

In [71]:
engine_local = create_engine('sqlite:///trade_ai.db')

In [72]:
#read from db
def fn_read_from_db(table_name, columns=None, where=None):
    print(engine_local)
    # SELECT cümlesi oluştur
    cols = ", ".join(columns) if columns else "*"
    sql = f"SELECT {cols} FROM {table_name}"
    
    if where:
        sql += f" WHERE {where}"
    
    # pandas ile SQL sorgusu çalıştır
    df = pd.read_sql(sql, con=engine_local)
    return df

In [73]:
df_raw = fn_read_from_db('tbl_ticker_actual_daily_2')

Engine(sqlite:///trade_ai.db)


In [74]:
# TICKER = 'ULKER.IS'

In [75]:
# df_ticker = df_raw[df_raw['TICKER']=='ULKER.IS']

In [76]:
def calculate_all_ranges(df, ticker, interval,CUTT_OFF=None):
    df = df.copy()
    df = df[df['TICKER']==ticker]
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])

    if CUTT_OFF is not None:
        # 1. string → datetime (dayfirst=True)
        cut_dt = pd.to_datetime(CUTT_OFF, dayfirst=True)
    
        # 2. df['DATETIME'] timezone-aware ise CUTT_OFF'u aynı tz yap
        if df['DATETIME'].dt.tz is not None:
            cut_dt = cut_dt.tz_localize(df['DATETIME'].dt.tz)
    
        # 3. Filtrele
        df = df[df['DATETIME'] <= cut_dt]

    # En yeniden eskiye sırala
    df = df.sort_values('DATETIME', ascending=False).reset_index(drop=True)

    # Eğer filtre sonrası boşsa direkt boş dön
    if df.empty:
        return pd.DataFrame()

    end_datetime = df.iloc[0]['DATETIME']

    range_map = {
        "1day": pd.DateOffset(days=1),
        "1week": pd.DateOffset(weeks=1),
        "1months": pd.DateOffset(months=1),
        "3months": pd.DateOffset(months=3),
        "6months": pd.DateOffset(months=6),
        "1year": pd.DateOffset(years=1),
        "2year": pd.DateOffset(years=2)
    }

    results = []

    for range_name, offset in range_map.items():

        range_startdate = end_datetime - offset

        df_filter = df[
            (df['DATETIME'] >= range_startdate) &
            (df['DATETIME'] <= end_datetime)
        ].copy()

        if df_filter.empty:
            continue

        range_size = len(df_filter)

        highest_idx = df_filter['HIGH'].idxmax()
        highest_value = round(df_filter.loc[highest_idx, 'HIGH'], 2)
        highest_date = df_filter.loc[highest_idx, 'DATETIME']

        # Highest sonrası (artık CUTT_OFF kontrolü gerekmiyor)
        df_after = df_filter[df_filter['DATETIME'] >= highest_date]
        row_size_after_highest = len(df_after)

        end_datetime_filtered = df_filter['DATETIME'].max()

   

        results.append({
            "TICKER": ticker,
            "INTERVAL":interval,
            "RANGE_TYPE": range_name,
            "RANGE_STARTDATE": range_startdate,
            "RANGE_SIZE": range_size,
            "HIGHEST_VALUE": highest_value,
            "HIGHEST_DATE": highest_date,
            "ROW_SIZE_A_H": row_size_after_highest,
            "CUTT_OFF_DATE":CUTT_OFF,
            "END_DATETIME": end_datetime_filtered
        })

    df_res = pd.DataFrame(results)
    return df_res

In [77]:
import pandas as pd
import numpy as np

def calculate_tv_frvp_best(ticker, df_ohlc, df_dates, value_area_pct=68):
    """
    TradingView FRVP mantığına uygun hesaplama.

    ticker: İşlem yapmak istediğin TICKER
    df_ohlc: OHLCV verisi (DATETIME, TICKER, OPEN, HIGH, LOW, CLOSE, VOLUME)
    df_dates: FRVP başlangıç tarihleri (TICKER, HIGHEST_DATE)
    value_area_pct: Value Area yüzdesi (TradingView parametresi, default=68)

    Döndürür: df_dates ile FRVP POC, VAL, VAH eklenmiş hali
    """
    
    # df_dates'den sadece ilgili ticker'ı filtrele
    df_ticker_dates = df_dates[df_dates['TICKER'] == ticker].copy()
    if df_ticker_dates.empty:
        raise ValueError(f"{ticker} için başlangıç tarihi bulunamadı.")
    
    # df_ohlc'de sadece ilgili ticker'ı al
    df_ohlc_ticker = df_ohlc[df_ohlc['TICKER'] == ticker].copy()
    if df_ohlc_ticker.empty:
        raise ValueError(f"{ticker} için OHLC verisi bulunamadı.")
    
    # DATETIME sütununu datetime tipine çevir
    df_ohlc_ticker['DATETIME'] = pd.to_datetime(df_ohlc_ticker['DATETIME'])
    
    # Çıktı df
    df_result = df_ticker_dates.copy()
    df_result['FRVP_POC'] = np.nan
    df_result['FRVP_VAL'] = np.nan
    df_result['FRVP_VAH'] = np.nan
    
    # Her HIGHEST_DATE için FRVP hesapla
    for idx, row in df_ticker_dates.iterrows():
        start_date = pd.to_datetime(row['HIGHEST_DATE'])
        
        # HIGHEST_DATE'ten itibaren veri al
        df_period = df_ohlc_ticker[df_ohlc_ticker['DATETIME'] >= start_date].copy()
        if df_period.empty:
            continue
        
        # Fiyat aralığı
        min_price = df_period['LOW'].min()
        max_price = df_period['HIGH'].max()
        price_bins = np.arange(min_price, max_price + 1, 1)  # Row Size = 1 tick
        
        # Volume profile histogram
        hist = pd.Series(0, index=price_bins)
        for _, bar in df_period.iterrows():
            low = bar['LOW']
            high = bar['HIGH']
            vol = bar['VOLUME']
            overlap = price_bins[(price_bins >= low) & (price_bins <= high)]
            if len(overlap) > 0:
                hist.loc[overlap] += vol / len(overlap)
        
        # POC
        poc = hist.idxmax()
        
        # Value Area (VAL, VAH)
        total_volume = hist.sum()
        target_volume = total_volume * (value_area_pct / 100)
        
        hist_sorted = hist.sort_index()
        cum_vol = hist_sorted.copy()
        cum_vol[:] = 0
        
        cum_vol[poc] = hist_sorted[poc]
        lower_idx = hist_sorted.index.get_loc(poc) - 1
        upper_idx = hist_sorted.index.get_loc(poc) + 1
        cum_total = hist_sorted[poc]
        
        while cum_total < target_volume:
            lower_vol = hist_sorted.iloc[lower_idx] if lower_idx >= 0 else 0
            upper_vol = hist_sorted.iloc[upper_idx] if upper_idx < len(hist_sorted) else 0
            
            if lower_vol >= upper_vol:
                cum_total += lower_vol
                cum_vol.iloc[lower_idx] = lower_vol
                lower_idx -= 1
            else:
                cum_total += upper_vol
                cum_vol.iloc[upper_idx] = upper_vol
                upper_idx += 1
            
            if lower_idx < 0 and upper_idx >= len(hist_sorted):
                break
        
        val = cum_vol[cum_vol > 0].index.min()
        vah = cum_vol[cum_vol > 0].index.max()
        
        # Sonuçları df_result'e ekle
        df_result.loc[idx, 'FRVP_POC'] = round(poc,2)
        df_result.loc[idx, 'FRVP_VAL'] = round(val,2)
        df_result.loc[idx, 'FRVP_VAH'] = round(vah,2)
    
    return df_result



In [ ]:
liste = [
    "AMZN",
    "AAPL",
    "DOAS.IS",
    # "NVDA",
    # "NFLX",
    # "ULKER.IS",
    # "ASELS.IS",
    # "CIMSA.IS",
    # "ASUZU.IS",
    # "ALKA.IS",
    # "GSRAY.IS",
    # "SASA.IS",
    # "ADESE.IS"
]
for TICKER in liste:
    print(TICKER)

    try:

        df_res = calculate_all_ranges(df_raw, TICKER,'daily', CUTT_OFF=None)
        df_out = fn_frvp_calc(SOURCE_TABLE = "tbl_ticker_actual_daily_2",
                    TICKER=TICKER,
                    INTERVAL = 'daily',
                    VALUE_AREA_PERC =  0.68,
                    PRICE_BIN = 1.0,
                    CUT_OFF=None)
        df_out.to_sql(
            name='test_frvp_poc',
            con=engine_local,
            if_exists='replace',
            index=False,
            # method="multi",  # performans için
            chunksize=500
        )
    except Exception as e:
        print(TICKER, e)

AMZN
Engine(sqlite:///trade_ai.db)
AMZN '>=' not supported between instances of 'Timestamp' and 'str'
AAPL
Engine(sqlite:///trade_ai.db)
AAPL '>=' not supported between instances of 'Timestamp' and 'str'
DOAS.IS
Engine(sqlite:///trade_ai.db)
DOAS.IS (sqlite3.OperationalError) table test_frvp_poc has no column named RANGE
[SQL: INSERT INTO test_frvp_poc ("TICKER", "INTERVAL", "RANGE", "START_DATETIME", "HIGH_DATETIME", "HIGH_VALUE", "HIGH_ROW_ID", "END_DATETIME", "ROW_COUNT_AFTER_HIGH", "FRVP_JSON", "POC", "VAL", "VAH", "PRICE_BIN", "VAL_PERC", "ROW_ID_FRVP", "RUNTIME") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [('DOAS.IS', 'daily', '1months', '2026-01-12 00:00:00.000000', '2026-02-10 00:00:00+03:00', 243.10000610351562, 'ID_DOAS.IS_20260210_0000_daily', '2026-02-11 00:00:00.000000', 2, '{"232.0": 381883.4923076923, "233.0": 381883.4923076923, "234.0": 381883.4923076923, "235.0": 381883.4923076923, "236.0": 261790.6923076923, "237.0": ... (180 characters t

In [84]:
#calc FRVP
df_res = fn_frvp_calc(SOURCE_TABLE = "tbl_ticker_actual_daily_2",
                TICKER='ULKER.IS',
                INTERVAL = 'daily',
                VALUE_AREA_PERC =  0.68,
                PRICE_BIN = 1.0,
                CUT_OFF=None)
df_res

Engine(sqlite:///trade_ai.db)


,TICKER,INTERVAL,RANGE,START_DATETIME,HIGH_DATETIME,HIGH_VALUE,HIGH_ROW_ID,END_DATETIME,ROW_COUNT_AFTER_HIGH,FRVP_JSON,POC,VAL,VAH,PRICE_BIN,VAL_PERC,ROW_ID_FRVP,RUNTIME
0,ULKER.IS,daily,1months,2026-01-12 00:00:00+03:00,2026-02-04 00:00:00+03:00,139.500000,ID_ULKER.IS_20260204_0000_daily,2026-02-11 00:00:00+03:00,6,"{""134.0"": 5263005.252380953, ""135.0"": 4445364....",134.323975,132.0,137.0,1.0,0.68,ID_ULKER.IS_20260204_0000_daily_1months,12.02.2026 - 18:31
1,ULKER.IS,daily,3months,2025-11-13 00:00:00+03:00,2026-02-04 00:00:00+03:00,139.500000,ID_ULKER.IS_20260204_0000_daily,2026-02-11 00:00:00+03:00,6,"{""134.0"": 5263005.252380953, ""135.0"": 4445364....",134.323975,132.0,137.0,1.0,0.68,ID_ULKER.IS_20260204_0000_daily_3months,12.02.2026 - 18:31
2,ULKER.IS,daily,6months,2025-08-15 00:00:00+03:00,2026-02-04 00:00:00+03:00,139.500000,ID_ULKER.IS_20260204_0000_daily,2026-02-11 00:00:00+03:00,6,"{""134.0"": 5263005.252380953, ""135.0"": 4445364....",134.323975,132.0,137.0,1.0,0.68,ID_ULKER.IS_20260204_0000_daily_6months,12.02.2026 - 18:31
3,ULKER.IS,daily,1year,2025-02-11 00:00:00+03:00,2026-02-04 00:00:00+03:00,139.500000,ID_ULKER.IS_20260204_0000_daily,2026-02-11 00:00:00+03:00,6,"{""134.0"": 5263005.252380953, ""135.0"": 4445364....",134.323975,132.0,137.0,1.0,0.68,ID_ULKER.IS_20260204_0000_daily_1year,12.02.2026 - 18:31
4,ULKER.IS,daily,2year,2024-02-12 00:00:00+03:00,2024-07-16 00:00:00+03:00,174.639227,ID_ULKER.IS_20240716_0000_daily,2026-02-11 00:00:00+03:00,399,"{""168.0"": 1806455.138888889, ""169.0"": 1806455....",113.775595,101.0,116.0,1.0,0.68,ID_ULKER.IS_20240716_0000_daily_2year,12.02.2026 - 18:31
